In [ ]:
from __future__ import division
import cv2
import numpy as np
import matplotlib.pyplot as plt

cap = cv2.VideoCapture(0)
kernel = np.ones((5,5),np.uint8)

kerneld = np.ones((4,4),np.uint8)
kernele = np.ones((5,5),np.uint8)
kerneleg = np.ones((4,4),np.uint8)
while(1):

    # Captura un frame
    _, frame = cap.read()

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # define range of blue color in HSV
    lower_green = np.array([45,80,80])
    upper_green = np.array([80,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_green, upper_green)
    #maskd = cv2.dilate(mask,kernel,iterations = 1) #1
    #maskd = cv2.erode(mask,kernel,iterations = 1) #2
    #maskd = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) # 3 (erotion and dilation)
    #maskd = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # 4 (dilation and erotion)
    
    #maskd = cv2.dilate(mask,kerneld,iterations = 1) #5
    #maskd = cv2.erode(maskd,kernele,iterations = 1) #5
    
    maskd = cv2.medianBlur(mask,11)
    #maskd = cv2.dilate(maskd,kernel,iterations = 1) #1
    maskd = cv2.morphologyEx(maskd, cv2.MORPH_GRADIENT, kernel) # 6

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= maskd)

    #cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('maskd',maskd)
    #cv2.imshow('res',res)
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()